In [ ]:
import torch
import networkx as nx
import numpy as np
from scipy.optimize import minimize
import time

# ===== your existing helper stays the same =====
class EarlyExit(Exception):
    pass

def MIS_checker_efficient_3(X_torch, adjacency_matrix_tensor, adjacency_matrix_tensor_comp):
    n_local = X_torch.shape[0]
    X_torch_binarized = X_torch.bool().float()
    X_torch_binarized_update = X_torch_binarized - 0.1*(-torch.ones(n_local) + (n_local*adjacency_matrix_tensor)@X_torch_binarized)
    X_torch_binarized_update = torch.clamp(X_torch_binarized_update, 0, 1)
    if torch.equal(X_torch_binarized, X_torch_binarized_update):
        MIS = torch.nonzero(X_torch_binarized).squeeze()
        return True, MIS
    return False, None

# ===== experiment settings (same as yours) =====
n = 500
p = 0.5
gamma_c = 1
beta = 0.8
alpha = 0.0001
ITERATION_T = 15000
NUM_TRIALS = 10
SEEDS = range(50)   # <- run 50 seeds

# for across-seed summary
best_size_per_seed = []
best_trial_per_seed = []
mean_runtime_per_seed = []
mean_iters_per_seed = []

for seed in SEEDS:
    # ---- build graph for this seed ----
    G = nx.gnp_random_graph(n, p, seed=seed)
    complement_G = nx.complement(G)

    num_components = nx.number_connected_components(G)
    print(f"\n================== Seed {seed} ==================")
    print("Number of connected components:", num_components)

    A = nx.adjacency_matrix(G)
    A_dense = A.todense()
    A_t = torch.tensor(A_dense, dtype=torch.float32)

    Acomp = nx.adjacency_matrix(complement_G)
    Acomp_dense = Acomp.todense()
    Acomp_t = torch.tensor(Acomp_dense, dtype=torch.float32)

    # gamma depends on complement degree
    degrees_c = dict(complement_G.degree())
    max_degree = max(degrees_c.values())
    gamma = 2 + max_degree

    # degree-based init d
    degrees = dict(G.degree())
    max_degree_node = max(degrees.values())
    d = torch.zeros(n)
    for node, degree in G.degree():
        d[node] = 1 - (degree / max_degree_node)
    d = d / d.max()

    # H and smooth objective/grad for SciPy
    H_np = (gamma * A_t - gamma_c * Acomp_t).numpy()

    def f_np(x):
        return -np.sum(x) + 0.5 * x.dot(H_np.dot(x))

    def grad_np(x):
        return -np.ones_like(x) + H_np.dot(x)

    bounds = [(0.0, 1.0)] * n

    # per-seed collectors
    MIS_size, Iter_NO, Run_time = [], [], []

    # same exploration noise as before
    exploration_parameter_eta = 2.0
    covariance_matrix = exploration_parameter_eta * torch.eye(len(d))

    print(f"ER({n}, {p}, seed={seed}): L-BFGS-B with MIS-check early exit")
    for init in range(NUM_TRIALS):
        if init == 0:
            x0_np = d.numpy()
        else:
            # make noise depend on (seed, init) so different seeds explore differently
            torch.manual_seed((seed << 16) + init)
            noise = torch.normal(mean=d, std=torch.sqrt(torch.diag(covariance_matrix)))
            x0_np = noise.numpy()

        iter_counter = {"count": 0}
        last_x = {"xk": None}

        def lbfgs_callback(xk):
            iter_counter["count"] += 1
            last_x["xk"] = xk.copy()
            x_t = torch.tensor(xk, dtype=torch.float32)
            checker, mis = MIS_checker_efficient_3(x_t, A_t, Acomp_t)
            if checker:
                raise EarlyExit()

        start_time = time.time()
        try:
            res = minimize(
                fun      = f_np,
                x0       = x0_np,
                method   = "L-BFGS-B",
                jac      = grad_np,
                bounds   = bounds,
                callback = lbfgs_callback,
                options  = {
                    "maxiter": ITERATION_T,
                    "gtol":    0.0,
                    "ftol":    0.0,
                    # "maxcor": 20,  # optional
                }
            )
            total_iters = res.nit
            x_final_np = res.x
        except EarlyExit:
            total_iters = iter_counter["count"]
            x_final_np = last_x["xk"]

        x_final = torch.tensor(x_final_np, dtype=torch.float32)
        checker, mis = MIS_checker_efficient_3(x_final, A_t, Acomp_t)

        elapsed = time.time() - start_time

        if mis is not None:
            print(f"init {init:2d} → MIS size {len(mis):3d}, iters {total_iters:5d}, time {elapsed:7.4f}s")
            MIS_size.append(len(mis))
            Iter_NO.append(total_iters)
            Run_time.append(elapsed)
        else:
            print(f"init {init:2d} → no MIS flagged by checker (iters {total_iters}, time {elapsed:7.4f}s)")

    # per-seed summary (printed before moving to next seed)
    if MIS_size:
        mis_sizes   = np.array(MIS_size)
        runtimes    = np.array(Run_time)
        iter_counts = np.array(Iter_NO)

        best_size = mis_sizes.max()
        best_run  = mis_sizes.argmax()
        mean_rt   = runtimes.mean()
        mean_it   = iter_counts.mean()

        p2_5_rt, p97_5_rt = np.percentile(runtimes, [2.5, 97.5])
        p2_5_it, p97_5_it = np.percentile(iter_counts, [2.5, 97.5])

        print(f"— Seed {seed} summary —")
        print(f"Best MIS size: {best_size} (trial {best_run})")
        print(f"Runtime (s): mean={mean_rt:.4f}, min={runtimes.min():.4f}, max={runtimes.max():.4f}, "
              f"std={runtimes.std(ddof=1):.4f}, 95% CI=({p2_5_rt:.4f}, {p97_5_rt:.4f})")
        print(f"Iters: mean={mean_it:.1f}, min={iter_counts.min()}, max={iter_counts.max()}, "
              f"std={iter_counts.std(ddof=1):.1f}, 95% CI=({p2_5_it:.1f}, {p97_5_it:.1f})")

        # store minimal across-seed info
        best_size_per_seed.append(best_size)
        best_trial_per_seed.append(int(best_run))
        mean_runtime_per_seed.append(float(mean_rt))
        mean_iters_per_seed.append(float(mean_it))
    else:
        print(f"— Seed {seed} summary — no MIS flagged by checker.")
        best_size_per_seed.append(None)
        best_trial_per_seed.append(None)
        mean_runtime_per_seed.append(None)
        mean_iters_per_seed.append(None)

# optional: across-seeds summary
vals = [v for v in best_size_per_seed if v is not None]
if vals:
    vals = np.array(vals)
    print("\n=========== Across-seeds summary (best per seed) ===========")
    print(f"Seeds with MIS: {len(vals)}/{len(SEEDS)}")
    print(f"Best MIS size: max={vals.max()}, min={vals.min()}, mean={vals.mean():.2f}, std={vals.std(ddof=1):.2f}")


================== Seed 0 ==================
Number of connected components: 1
ER(500, 0.5, seed=0): L-BFGS-B with MIS-check early exit
init  0 → MIS size  11, iters   150, time  1.0069s
init  1 → MIS size  10, iters   137, time  1.0463s
init  2 → MIS size  11, iters   135, time  1.3770s
init  3 → MIS size  10, iters   172, time  2.0073s
init  4 → MIS size   9, iters   184, time  1.8430s
init  5 → MIS size  12, iters   134, time  0.5880s
init  6 → MIS size  10, iters   151, time  0.6278s
init  7 → MIS size   9, iters   158, time  0.7408s
init  8 → MIS size   9, iters   165, time  0.8299s
init  9 → MIS size  13, iters   198, time  0.9642s
— Seed 0 summary —
Best MIS size: 13 (trial 9)
Runtime (s): mean=1.1031, min=0.5880, max=2.0073, std=0.4909, 95% CI=(0.5970, 1.9704)
Iters: mean=158.4, min=134, max=198, std=21.5, 95% CI=(134.2, 194.8)

================== Seed 1 ==================
Number of connected components: 1
ER(500, 0.5, seed=1): L-BFGS-B with MIS-check early exit
init  0 → MIS 